# oven temperature determination

In [105]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as spo


In [106]:
#----pressure and effusion functions------
ovenHoleDiam=1e-3
ovenHoleArea=np.pi*(ovenHoleDiam/2)**2 #SI
m_Li=1.1623e-26 #kg
K_B=1.3806e-23 #botlzman, SI

def pressure_ABC(T,A,B,C):
    return 10**(A-(B/(T+C)))
@np.vectorize
def pressure_NIST(T):
    P1,P2=np.nan,np.nan
    if 298.14<=T<=1599.99:
        A,B,C=4.98831,7918.984,-9.52
        P1=pressure_ABC(T,A,B,C)
    if 1204<=T<=1353:
        A,B,C=1.58992,1603.966,-711.088
        P2=pressure_ABC(T,A,B,C)
    P_Bar=np.nanmean([P1,P2])  #these values agree
    assert np.isnan(P_Bar)==False
    P_Pa=100_000*P_Bar
    return P_Pa
def Q_Effusive(T,A=ovenHoleArea,m=m_Li):
    P=pressure_NIST(T)
    return P*A/np.sqrt(2*np.pi*m*K_B*T)
@np.vectorize()
def peak_Signal_Fit(deltaT,T0,S0):
    return S0*pressure_NIST(T0+deltaT)/np.sqrt(T0+deltaT)

### Data
To fit the temperature function I need data in the form of the peak signal minus the background. I will keep it simple and use the fits file to get this. This works if I have the laser in the same place for a variety of temperatures